In [ ]:
import os
import math
import numpy as np

from collada import Collada

from src import tools

In [ ]:
# Pull data changes
! gdown https://drive.google.com/drive/folders/1paJR2dbe9fIMmVpEnY4_308gpfAnP-l4?usp=share_link --folder

## WeFish Data

In [ ]:
## Traverse Data

data_root = "data/WeFish/raw/"

file_paths = []
for mesh_d in os.listdir(data_root):
    mesh_d_root = os.path.join(data_root, mesh_d)
    dae_files = filter(lambda x: x.endswith(".dae"), os.listdir(mesh_d_root))
    for f_name in dae_files:
        file_paths.append(os.path.join(mesh_d_root, f_name))
        
print(f"Found {len(file_paths)} files")

In [ ]:
## Visualize shapes

traces = []
for idx in np.random.choice(len(file_paths), 3):
    mesh = Collada(file_paths[idx])

    sub_traces = []

    for obj in mesh.geometries:
        verts = obj.primitives[0].vertex
        faces = obj.primitives[0].indices[:, :, 0]

        sub_traces += tools.plot_3d_mesh(verts, faces)
    traces.append(sub_traces)

tools.show_grid(*traces)

### Process shapes

In [ ]:
# Read all
meshes_verts, meshes_faces = [], []
for path in file_paths:
    mesh = Collada(path)

    meshes_verts.append(mesh.geometries[0].primitives[0].vertex)
    meshes_faces.append(mesh.geometries[0].primitives[0].indices[:, :, 0])

# Min-max center every shape
meshes_verts = [x - (x.min(axis=0) + x.max(axis=0)) / 2 for x in meshes_verts]
    
# Calculate the Bboxes
bbox_min = np.concatenate(meshes_verts).min(axis=0)
bbox_max = np.concatenate(meshes_verts).max(axis=0)

print(f"Bbox coords from {bbox_min} to {bbox_max}")

scale_factor = math.ceil((bbox_max - bbox_min).max() / 2)
print(f"Scale factor: {scale_factor}")

# Scale everyone to fit [-1, 1] box along the longest dimension
meshes_verts = [x / scale_factor for x in meshes_verts]

In [ ]:
## Visualize processed shapes

traces = []
for idx in np.random.choice(len(file_paths), 3):
    verts = meshes_verts[idx]
    faces = meshes_faces[idx]

    traces.append(tools.plot_3d_mesh(verts, faces))

tools.show_grid(*traces)

## Save Shapes

In [ ]:
save_root = "data/WeFish/converted/"

for idx in range(len(meshes_verts)):
    igl.write_obj(
        os.path.join(save_root, f"{idx:04d}.stl"),
        meshes_verts[idx],
        meshes_faces[idx])

In [ ]:
! zip -r converted.zip {save_root} 